In [1]:
from keras.applications.vgg16 import VGG16
import matplotlib.image as mpimg
from keras import backend as K
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import cv2
from keras.applications.vgg16 import decode_predictions

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
    
def cam(img_path, model):    
    #     %matplotlib inline
    K.set_learning_phase(0)
    img=mpimg.imread(img_path)
#     plt.imshow(img)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    argmax = np.argmax(preds[0])
    output = model.output[:, argmax]
    last_conv_layer = model.get_layer('block5_conv3')
    grads = K.gradients(output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    for i in range(512):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    import cv2
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    hif = .8
    output_heatM = heatmap * hif + img
    #output = 'output.jpeg'
    output = 'hmpCAM/' + imagesFileNames[imgCount]
    
#     cv2.imwrite(output, output_heatM) superimosed
    cv2.imwrite(output, heatmap)        
    return None

Using TensorFlow backend.


In [2]:
import cv2
import os

folder_Images="ImagentDataset"

# last_fileName=""
# images = []
imagesFileNames = []
for filename in os.listdir(folder_Images):
    imagesFileNames.append(filename)
imagesFileNames.sort()  

In [3]:
import cv2
import os


folder_Bboxes="ILSVRC2012_bbox_val_v3_All"

bbox_FileNames = []
for filename in os.listdir(folder_Bboxes):
    bbox_FileNames.append(filename)
bbox_FileNames.sort()  

In [4]:
import xml.etree.ElementTree as ET

def get_ClassId(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    class_Id = ""

    for names in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        classId = str(names.find("name").text)

    return classId

In [5]:
import xml.etree.ElementTree as ET

def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes

In [ ]:
imgCount=0
hmpCount =0
K.clear_session()


model = VGG16(weights='imagenet', include_top=True)

for imgFileName in imagesFileNames:
    if(imgCount==20):
        K.clear_session()
        ## the function cam(imgFileName, model) can be directly used here withought all the code below, it was just extracted here 
        ## for some code optimization issues
        model = VGG16(weights='imagenet', include_top=True)
    name, boxes = read_content("ILSVRC2012_bbox_val_v3"+ "/" + bbox_FileNames[imgCount])
    if len(boxes)==1:      
        K.set_learning_phase(0)
        img_path = 'ImagentDataset/' + imagesFileNames[imgCount]
        img=mpimg.imread(img_path)
    #     plt.imshow(img)
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x)
        argmax = np.argmax(preds[0])
        output = model.output[:, argmax]
        last_conv_layer = model.get_layer('block5_conv3')
        grads = K.gradients(output, last_conv_layer.output)[0]
        pooled_grads = K.mean(grads, axis=(0, 1, 2))
        iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
        pooled_grads_value, conv_layer_output_value = iterate([x])
        for i in range(512):
            conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
        heatmap = np.mean(conv_layer_output_value, axis=-1)
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)
        import cv2
        img = cv2.imread(img_path)
        heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        hif = .8
        output_heatM = heatmap * hif + img
        #output = 'output.jpeg'
        output = 'hmpCAM/' + imagesFileNames[imgCount]

    #     cv2.imwrite(output, output_heatM) superimosed
        cv2.imwrite(output, heatmap)      
#         cam('ImagentDataset/' + imagesFileNames[imgCount], model)
        print('image count', imgCount)
        hmpCount+=1
    imgCount+=1

    
    

image count 1
image count 2
image count 3
image count 4
image count 5
image count 6
